# Tune GPT2 to generate controlled sentiment reviews
> Optimise GPT2 to produce IMDB movie reviews with controlled sentiment using a BERT sentiment classifier for rewards.

**WARNING:** We often experienced loss spikes in this examples which caused model training to fail or slow down. There is a [GitHub issue](https://github.com/lvwerra/trl/issues/101) to track the issue.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2-ctrl-training-setup.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


The experiment setup is very similar to the positive sentiment notebook. However, in this notebook we fine-tune GPT2 (small) to generate **controlled** movie reviews based on the IMDB dataset. The model gets the target sentiment and 5 tokens from a real review and is tasked to produce continuations with the targeted sentiment. The reward for the continuations is calculated with the logits of a BERT sentiment classifier. That reward is then used for PPO training.

## Setup experiment

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import torch
import wandb
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from random import choices
import matplotlib.pyplot as plt
import warnings
from pathlib import Path
from datasets import Dataset
import warnings

tqdm.pandas()

from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model

/home/diogo/ASH_code/trl-venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuration

In [3]:
sentiment_pipe_kwargs = {"top_k": None, "function_to_apply": "none"}

config = PPOConfig(
    model_name="lvwerra/gpt2-imdb", steps=51200, learning_rate=1.41e-5, remove_unused_columns=False, log_with="wandb"
)

# config = PPOConfig(
#     model_name="gpt2-large", steps=51200, learning_rate=1.41e-5, remove_unused_columns=False, log_with="wandb"
# )

txt_in_len = 8
txt_out_len = 24
seed = 1
np.random.seed(seed)

fatal: No names found, cannot describe anything.


You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [4]:
gpt2_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
gpt2_model_ref = create_reference_model(gpt2_model)
gpt2_tokenizer = AutoTokenizer.from_pretrained(config.model_name)
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 500 characters long and take the first 1000 characters of each comment. The first filter we apply to avoid comments that are less than `txt_in_len` token long and the second to avoid tokenizing way more text than we actually need.

In [5]:
toy = 1
rate = '0.2'
path = os.path.join(Path.home(), "nlp_data", f"imdb_{toy}")
file_dict = {
    "train" : os.path.join(path,"finetune_{}_train.tsv".format(rate))
}
dataset = load_dataset('csv',
                        data_files=file_dict,
                        delimiter='\t'
            )
dataset = dataset['train']
dataset = dataset.map(lambda x: {"label": 'P' if x["label"] else 'N'}, batched=False)

dataset

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1236.16it/s]
Generating train split: 90000 examples [00:00, 634549.85 examples/s]
Map: 100%|██████████| 90000/90000 [00:04<00:00, 22355.06 examples/s]


Dataset({
    features: ['review', 'label', 'section'],
    num_rows: 90000
})

### Tokenize IMDB reviews

We tokenize all IMDB in advance to avoid tokenizing twice. In the first step we encode the queries and slice the first `txt_in_len` tokens. In a second step we decode these tokens back to text for later display.

In [6]:
dataset = dataset.map(
    lambda x: {"input_ids": gpt2_tokenizer.encode(x["review"], return_tensors="pt")[0, :txt_in_len]},
    batched=False,
)
dataset = dataset.map(lambda x: {"query": gpt2_tokenizer.decode(x["input_ids"])}, batched=False)
dataset = dataset[:20480] # Don't know why this is here

dataset = Dataset.from_dict(dataset)
dataset.set_format("pytorch")

Map:   1%|          | 1000/90000 [00:00<00:19, 4485.74 examples/s]

Map: 100%|██████████| 90000/90000 [00:11<00:00, 8175.51 examples/s]


In [7]:
dataset[3]["input_ids"]

tensor([   40,   481,   307,  5508,    11,  1312, 26399,   428])

In [8]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [9]:
ppo_trainer = PPOTrainer(config, gpt2_model, gpt2_model_ref, gpt2_tokenizer, dataset, data_collator=collator)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: diogocruz. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [10]:
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
else:
    device = ppo_trainer.accelerator.device
sentiment_pipe = pipeline("sentiment-analysis", "lvwerra/distilbert-imdb", device=device)
# sentiment_pipe = pipeline("sentiment-analysis", "textattack/xlnet-base-cased-imdb", device=device)

The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

The resulting reward signal:

In [11]:
def extract_pipe_output(outputs):
    positive_logits = []
    for out in outputs:
        for element in out:
            if element["label"] == "POSITIVE":
                positive_logits.append(torch.tensor(element["score"]))
    return positive_logits

### Control token dict
We will append the control token at the beginning of each query to signal the model what the target sentiment is. Each control sequence consists of three tokens:

In [12]:
# ctrl_str = ["Movie review:", "Film review:"]
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # this should be handled by accelerate
# ctrl_tokens = dict((s, gpt2_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)

In [13]:
# ctrl_tokens

### Reward function

In [14]:
def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [neutral]: reward = -2*abs(logit)+4
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i] == 'N':
            logit[i] = -logit[i]
        # elif task[i] == "[neutral]":
        #     logit[i] = -2 * torch.abs(logit[i]) + 4
        elif task[i] == 'P':
            pass
        else:
            raise ValueError("task has to be in [0, 1, 2]!")
    return logit

The following examples show the rewards for the cases where the classifier logit is 4, -4 and 0 for the three targets `['negative]`, `['neutral]` and `['positive']`. The scaling is not perfect as it differs between neutral and the other two classes. This is something to further investigate in the future. Ideally, one would use the logit output for each class individually, but since there is no dedicated class for neutral this is a workaround.

### Generation settings

In [15]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": gpt2_tokenizer.eos_token_id,
    "max_new_tokens": txt_out_len,
    "eos_token_id": -1,
}

## Optimize model

**Steps**

The training loop consists of the following steps:
1. Get a batch of queries and create random controls
2. Get the query responses from the policy
3. Join query and responses and tokenize for BERT analysis
4. Get sentiments for query/responses from BERT
5. Optimize policy with PPO using the (query, response, reward) triplet
6. Log all the training statistics

**Training time**

This step takes **~2h** on a P6000 GPU with the above specified settings.

In [16]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for epoch in range(1):
        for batch in tqdm(ppo_trainer.dataloader):
            (logs, game_data,) = (
                dict(),
                dict(),
            )

            #### prepend a random control token
            # task_list = choices(ctrl_str, k=config.batch_size)
            # game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]
            # query_tensors = [torch.cat((ctrl_tokens[t], input_ids)) for t, input_ids in zip(task_list, batch["input_ids"])]

            task_list = batch['label']
            game_data["query"] = batch["query"]
            query_tensors = batch["input_ids"]

            #### get response from gpt2
            response_tensors = []
            for query in query_tensors:
                response = ppo_trainer.generate(query, **generation_kwargs)
                response_tensors.append(response.squeeze()[-txt_out_len:])
            game_data["response"] = [gpt2_tokenizer.decode(r.squeeze()) for r in response_tensors]

            #### sentiment analysis
            texts = [q + r for q, r in zip(batch["query"], game_data["response"])]
            logits = extract_pipe_output(sentiment_pipe(texts, **sentiment_pipe_kwargs))
            rewards = pos_logit_to_reward(logits, task_list)

            #### Run PPO training
            #t = time.time()
            stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

            for cs in ['P','N']:
                key = "env/reward_" + cs
                stats[key] = np.mean([r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs])
            ppo_trainer.log_stats(stats, game_data, rewards)

  0%|          | 0/80 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
 38%|███▊      | 30/80 [54:30<1:30:50, 109.01s/it]

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the following:

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2-ctrl-training-stats.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher inital coefficient.

## Model inspection

### Reward distribution
First, we can have a look at the reward distribution. Both the negative and positive rewards are clearly shifted to high rewards. The neutral rewards, however, are still centered around zero. There are a few possible explanations for this. There could be a bug in the code and the way the neutral rewards are calculated. Another problem could be that sentence sometimes start with a strong sentiment and it is hard for the model shift the sentiment towards neutral.

## Save model
Finally, we save the model to disk for later usage.

In [ ]:
gpt2_model.save_pretrained(f"gpt2-sentiment_task{toy}_rate{rate}_seed{seed}")
gpt2_tokenizer.save_pretrained(f"gpt2-sentiment_task{toy}_rate{rate}_seed{seed}")

('gpt2-sentiment_task4_rate0.5_seed1/tokenizer_config.json',
 'gpt2-sentiment_task4_rate0.5_seed1/special_tokens_map.json',
 'gpt2-sentiment_task4_rate0.5_seed1/vocab.json',
 'gpt2-sentiment_task4_rate0.5_seed1/merges.txt',
 'gpt2-sentiment_task4_rate0.5_seed1/added_tokens.json',
 'gpt2-sentiment_task4_rate0.5_seed1/tokenizer.json')

# Test Model

In [ ]:
# config = PPOConfig(
#     model_name=f"gpt2-sentiment_task{toy}_rate{rate}_seed{seed}", steps=51200, learning_rate=1.41e-5, remove_unused_columns=False, log_with="wandb"
# )
# gpt2_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
# gpt2_model_ref = create_reference_model(gpt2_model)
# gpt2_tokenizer = AutoTokenizer.from_pretrained(config.model_name)
# gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

In [ ]:
path = os.path.join(Path.home(), "nlp_data", f"imdb_{toy}")
file_dict = {
    "test" : os.path.join(path,"test.tsv"),
    "strong" : os.path.join(path,"test_strong.tsv"),
    "weak" : os.path.join(path,"test_weak.tsv"),
    "both" : os.path.join(path,"test_both.tsv"),
    "neither" : os.path.join(path,"test_neither.tsv")
}
test_dataset = load_dataset('csv',
                        data_files=file_dict,
                        delimiter='\t'
            )
cases = ['test','strong','weak','both','neither']
for case in cases:
    test_dataset[case] = test_dataset[case].map(lambda x: {"label": 'P' if x["label"] else 'N'}, batched=False)
    test_dataset[case] = test_dataset[case].map(
        lambda x: {"input_ids": gpt2_tokenizer.encode(x["review"], return_tensors="pt")[0, :txt_in_len]},
        batched=False,
    )
    test_dataset[case] = test_dataset[case].map(lambda x: {"query": gpt2_tokenizer.decode(x["input_ids"])}, batched=False)
    test_dataset[case] = test_dataset[case][:20480] # Don't know why this is here

    test_dataset[case] = Dataset.from_dict(test_dataset[case])
    test_dataset[case].set_format("pytorch", device='cuda')

Extracting data files: 100%|██████████| 5/5 [00:00<00:00, 20.72it/s]
Generating test split: 7000 examples [00:00, 457779.22 examples/s]
Generating strong split: 1000 examples [00:00, 209788.63 examples/s]
Generating weak split: 1000 examples [00:00, 210589.14 examples/s]
Generating both split: 2519 examples [00:00, 240808.02 examples/s]
Generating neither split: 2481 examples [00:00, 226322.20 examples/s]
Map: 100%|██████████| 2481/2481 [00:00<00:00, 5525.63 examples/s]


In [ ]:
test_rewards = {}
stats = {}
for case in ['strong','weak','both','neither']:
    #stats[case] = {}

    data = test_dataset[case]

    task_list = data['label']
    # game_data["query"] = data["query"]
    query_tensors = data['input_ids']

    #### get response from gpt2
    response_tensors = []
    for query in query_tensors:
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-txt_out_len:])
    responses = [gpt2_tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### sentiment analysis
    texts = [q + r for q, r in zip(data["query"], responses)]
    logits = extract_pipe_output(sentiment_pipe(texts, **sentiment_pipe_kwargs))
    rewards = pos_logit_to_reward(logits, task_list)

    # for cs in ['P','N']:
    #     stats[case][cs] = np.mean([r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs])
    # for cs in ['P','N']:
    stats[case] = np.mean([r.cpu().numpy() for r in rewards])

/home/diogo/ASH_code/trl-venv/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
print(stats)

{'strong': -1.7826798, 'weak': 1.840446, 'both': -1.3883007, 'neither': 2.0656693}


In [22]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


env/reward_N,▁▃▂▃▃▃▄▄▅▄▄▅▄▅▅▅▄▅▅▄▅▅▅▅▅▅▅▅▆▆▇▆▆▅▆▇█▆▇▇
env/reward_P,█▇▇▅▆▇▅▅▅▅▅▅▅▅▅▄▅▅▄▄▅▅▅▅▅▆▅▅▃▅▃▅▅▅▅▃▁▄▃▂
env/reward_mean,▁▃▂▃▃▃▄▃▅▄▄▅▄▅▄▄▄▅▄▄▅▅▅▅▅▆▆▅▆▆▇▆▆▆▇▇█▇▆▇
env/reward_std,▃▁▂▁▂▁▃▂▁▂▂▁▂▄▃▃▃▂▄▃▂▂▁▂▃▁▃▂▄▃▅▃▂▃▃▅█▅▅▃
objective/entropy,▇▇███████████▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▄▅▄▄▃▃▁▂▂▁
objective/kl,▁▃▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇█▇▇▇▇
objective/kl_coef,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
ppo/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ppo/loss/policy,▆▆█▅▆▆▆▇▆▅▆▄▆▂▅▅▄▄▄▃▄▂▂▄▃▃▂▃▂▃▂▁▂▃▃▁▃▂▁▂
ppo/loss/total,▃▄▄▃▃▃▃▄▃▃▃▂▃▂▃▃▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▁▂▂▂█▃▃▂▂
ppo/loss/value,▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▂▁▁▂▁▃▁▂▁▂▁▁▂▂█▃▃▂▂


In [20]:
print(stats)

{'strong': -2.725019, 'weak': 1.9843187, 'both': -1.763271, 'neither': 2.795128}


In [56]:
print(stats)

{'strong': {'P': -2.6375723, 'N': nan}, 'weak': {'P': nan, 'N': -2.3930578}, 'both': {'P': 2.5258434, 'N': nan}, 'neither': {'P': nan, 'N': 2.6546242}}
